# Notebook Electricity Consumption  in Barcelona

> *Andrea Ramirez*
>
> Master in Data Science
>
> Universitat de Girona, 2023

# 1. Notebook Preparation

*Import the basic libraries to run this notebook.*

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from babel.dates import format_date
import json
from datetime import datetime, time
import os
import requests

# 2. Data Extraction

*Import of libraries to run Spark in this section.*

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from itertools import chain
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.functions import regexp_replace, to_date

from pyspark.ml.feature import *
from pyspark.ml import *

*Creation of the Spark session*

In [3]:
os.environ['SPARK_MASTER'] = 'local[*]'
os.environ['SPARK_JAR_PACKAGES'] = 'org.postgresql:postgresql:42.6.0'
os.environ['SPARK_WAREHOUSE'] = '/tmp/'

In [4]:
spark = SparkSession.builder.master(os.environ['SPARK_MASTER']) \
    .appName("Spark-Electricity-BCN") \
    .config("spark.sql.session.timeZone","UTC") \
    .config('spark.ui.enabled', False) \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .config('spark.driver.memory', '2g') \
    .config('spark.ui.showConsoleProgress', False) \
    .config('spark.jars.packages',os.environ['SPARK_JAR_PACKAGES']) \
    .getOrCreate()

*Getting the `url` per year*.

In [5]:
data_activities_BCN = {
    2019: {'year':'2019', 'url':'https://opendata-ajuntament.barcelona.cat/data/dataset/5a8ea22e-6e37-4867-99c0-ce2febe665b4/resource/09a35229-b075-447a-85d0-a868ed86fa51/download'},
    2020: {'year':'2020', 'url':'https://opendata-ajuntament.barcelona.cat/data/dataset/5a8ea22e-6e37-4867-99c0-ce2febe665b4/resource/26cd4ea5-0930-469a-925f-18cf066ae8cb/download'},
    2021: {'year':'2021', 'url':'https://opendata-ajuntament.barcelona.cat/data/dataset/5a8ea22e-6e37-4867-99c0-ce2febe665b4/resource/7a04129f-8aef-4d4b-9b9e-afe11c0360a4/download'},
    2022: {'year':'2022', 'url':'https://opendata-ajuntament.barcelona.cat/data/dataset/5a8ea22e-6e37-4867-99c0-ce2febe665b4/resource/0d3f939b-72f5-42fe-bb8e-70acfd1f65cf/download'},
}

In [6]:
def get_data_per_year(diccionary, element):
    if element in diccionary:
        data_per_year = diccionary[element]
        year = data_per_year['year']
        url = data_per_year['url']
        return year, url
    else:
        return None 

*Requesting the data from the Open Data website*

In [7]:
def extract_data(year, url):
    file_name = f"{year}.csv"
    response = requests.get(url)
    with open(file_name, "wb") as f:
        f.write(response.content)
    df = spark.read.csv(file_name, header=True, inferSchema=True)
    return df

In [8]:
combined_activities_df = None

for year, data_per_year in data_activities_BCN.items():
    year = data_per_year['year']
    url = data_per_year['url']
    activities_df = extract_data(year, url)
    if combined_activities_df is None:
        combined_activities_df = activities_df
    else:
        combined_activities_df = combined_activities_df.union(activities_df)

# 3. Data Exploration and Pre-Processing

In [9]:
activities_df = combined_activities_df.select('Any','Titularitat','Festa','DataInici','DataFi','Assistents','Espai')

First of all, let's check the initial attributes.

In [10]:
def dataframe_description(dataframe):
    data_types = dataframe.dtypes
    unique_values = [dataframe.select(col).distinct().count() for col in dataframe.columns]
    info_columns = spark.createDataFrame(list(zip(dataframe.columns, data_types, unique_values)), ["column", "data_types", "unique_values"])
    return info_columns

In [11]:
info_initial_columns = dataframe_description(activities_df)
info_initial_columns.show(truncate=False)

+-----------+---------------------+-------------+
|column     |data_types           |unique_values|
+-----------+---------------------+-------------+
|Any        |{Any, int}           |4            |
|Titularitat|{Titularitat, string}|3            |
|Festa      |{Festa, string}      |17           |
|DataInici  |{DataInici, string}  |57           |
|DataFi     |{DataFi, string}     |56           |
|Assistents |{Assistents, int}    |21           |
|Espai      |{Espai, string}      |22           |
+-----------+---------------------+-------------+



## 3.1. Data Cleaning

Let's clean up the data of this dataframe.

In [12]:
activities_df = activities_df.withColumnRenamed("Any", "year_observed") \
    .withColumnRenamed("Titularitat", "organizers_type") \
    .withColumnRenamed("Festa", "activity") \
    .withColumnRenamed("DataInici", "date_start") \
    .withColumnRenamed("DataFi", "date_end") \
    .withColumnRenamed("Assistents", "attendants_value") \
    .withColumnRenamed("Espai", "venue_type") 

In [13]:
activities_df = activities_df.withColumn("date_start", to_date(activities_df.date_start, "dd/MM/yyyy")) \
    .withColumn("date_end", to_date(activities_df.date_end, "dd/MM/yyyy")) 

There are some activities that are held for more than one day. For that reason, let's calculate the difference between `date_end` and `date_start`. Then, let's use the function `posexplode` to create an index for each range to finally duplicate the other values of the row.

In [14]:
activities_df = activities_df.withColumn("date_diff", expr("date_end - date_start"))

activities_df = activities_df.selectExpr("year_observed","activity","organizers_type","attendants_value","venue_type","date_start","date_diff","posexplode(sequence(date_start, date_end, interval 1 day)) as (pos, date_observed)")

activities_df = activities_df.select("year_observed","activity","organizers_type","attendants_value","venue_type", col("date_observed").alias("date_observed"))

In [15]:
info_initial_columns = dataframe_description(activities_df)
info_initial_columns.show(truncate=False)

+----------------+-------------------------+-------------+
|column          |data_types               |unique_values|
+----------------+-------------------------+-------------+
|year_observed   |{year_observed, int}     |4            |
|activity        |{activity, string}       |17           |
|organizers_type |{organizers_type, string}|3            |
|attendants_value|{attendants_value, int}  |21           |
|venue_type      |{venue_type, string}     |22           |
|date_observed   |{date_observed, date}    |159          |
+----------------+-------------------------+-------------+



Function to map dictionaries.

In [16]:
def mapping_strings(dataframe,mapping_dic,mapped_column,original_column):
    mapping = create_map([lit(x) for x in chain(*mapping_dic.items())])
    dataframe = dataframe.withColumn(mapped_column, when(col(original_column).isin(list(mapping_dic.keys())), mapping[col(original_column)]).otherwise(col(original_column)))
    return dataframe

In [17]:
map_years = {
    2019:"2019",
    2020:"2020",
    2021:"2021",
    2022:"2022",
    2023:"2023"
}
activities_df = mapping_strings(activities_df,map_years,"year_observed","year_observed")

In [18]:
unique_organizers_type = activities_df.select("organizers_type").distinct().collect()

for row in unique_organizers_type:
    print(row[0])

Municipal
Mixta
Privada


In [19]:
replace_organizers_type = {
    "Municipal":"Public",
    "Mixta":"Hybrid",
    "Privada":"Private"
}
activities_df = mapping_strings(activities_df,replace_organizers_type,"organizers_type","organizers_type")

In [20]:
unique_venue_type = activities_df.select("venue_type").distinct().collect()

for row in unique_venue_type:
    print(row[0])

Barri de Gràica, Sant Gervasi i la Bordeta
None
Museus de l'Institut de Cultura
Barri de Sant Antoni
Districte de Gràcia
Districte de Sants
Diversos espais
Diversos espais de la ciutat
Diferents espais de la ciutat
Gràcia
Ciutat i districtes
Diferents museus
Online
Ciutat Vella
Diferents espais
Diversos museus de la ciutat
100 llibreries de Catalunya (51 d'elles de Barcelona)
Diversos espais del barri de Gràcia
Diversos espais de Ciutat Vella
Avinguda de la Reina Maria Cristina
Diversos espais del barri de Sants
Diversos museus de la ciutat i de l'Àrea Metropolitana


In [21]:
replace_venue_type = {
"Barri de Gràica, Sant Gervasi i la Bordeta":"08012",
"None":"None",
"Museus de l'Institut de Cultura":"08002",
"Barri de Sant Antoni":"08001",
"Districte de Gràcia":"08012",
"Districte de Sants":"08014",
"Diversos espais":"Several places",
"Diversos espais de la ciutat":"Several places",
"Diferents espais de la ciutat":"Several places",
"Gràcia":"08012",
"Ciutat i districtes":"Several places",
"Diferents museus":"Several places",
"Online":"None",
"Ciutat Vella":"08002",
"Diferents espais":"Several places",
"Diversos museus de la ciutat":"Several places",
"100 llibreries de Catalunya (51 d'elles de Barcelona)":"Several places",
"Diversos espais del barri de Gràcia":"08012",
"Diversos espais de Ciutat Vella":"08002",
"Avinguda de la Reina Maria Cristina":"08004",
"Diversos espais del barri de Sants":"08014",
"Diversos museus de la ciutat i de l'Àrea Metropolitana":"Several places",
}
activities_df = mapping_strings(activities_df,replace_venue_type,"location","venue_type")
activities_df = activities_df.fillna("Several places", subset=["location"])

*Selecting the columns to work*.

In [22]:
drop_activities = ['activity','venue_type','attendants_value','year_observed']
df_activities = activities_df.drop(*drop_activities)

In [23]:
df_activities.show(1)

+---------------+-------------+--------------+
|organizers_type|date_observed|      location|
+---------------+-------------+--------------+
|         Public|   2019-12-31|Several places|
+---------------+-------------+--------------+
only showing top 1 row



In [26]:
df_activities.coalesce(1).write.csv('activities_bcn_2019_2022.csv', header=True)